In [6]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py

--2025-06-03 02:35:41--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2025-06-03 02:35:41 (53.0 MB/s) - ‘minsearch.py’ saved [3832/3832]



# SEARCH DOCs 

In [2]:
import minsearch

In [3]:
import json

### preprocess documents

In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
for course_dict in docs_raw:
    print(course_dict['course'])

data-engineering-zoomcamp
machine-learning-zoomcamp
mlops-zoomcamp


In [6]:
documents = []

for course_dict in docs_raw:
    for doc_dict in course_dict['documents']:
        doc_dict['course'] = course_dict['course']
        documents.append(doc_dict)

In [7]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

### applying min search

In [8]:
# indexing

index = minsearch.Index(
    text_fields = ["question", "text", "section"],
    keyword_fields = ["course"] 
)
# SELECT * WHERE course="data-engineering-zoomcamp"

In [9]:
index.fit(documents)

In [10]:
q = "the course is already started, can I still enroll?"

In [11]:
boost = {'question': 3.0, 'section': 0.5,}

result = index.search(
    query=q,
    filter_dict = {"course" : "data-engineering-zoomcamp"},
    boost_dict=boost,
    num_results=5
)

In [12]:
result

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

# Using GPT OPENAI API

In [13]:
from openai import OpenAI

client = OpenAI()

In [14]:
q

'the course is already started, can I still enroll?'

In [15]:
response = client.chat.completions.create(
    model = "gpt-4o",
    messages = [{"role": "user", "content": q}] 
)
response

ChatCompletion(id='chatcmpl-BlvN1OZaTrlBex01ib6XrkCAszt3h', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Whether you can still enroll in a course that has already started depends on several factors, including the policies of the institution offering the course, the specific course structure, and the level of flexibility permitted by the instructor. Here are some general steps you can take:\n\n1. **Contact the Instructor or Institution**: Reach out directly to the course instructor or the administration office to inquire if late enrollment is possible. They can provide specific information on whether you can join the course late and what steps you need to follow.\n\n2. **Check for an Online Option**: Some courses offer online resources, recorded lectures, or flexible deadlines that can accommodate late enrollments.\n\n3. **Understand the Requirements**: If you are allowed to enroll late, make sure you understand any missed material

In [16]:
print(response.choices[0].message.content)

Whether you can still enroll in a course that has already started depends on several factors, including the policies of the institution offering the course, the specific course structure, and the level of flexibility permitted by the instructor. Here are some general steps you can take:

1. **Contact the Instructor or Institution**: Reach out directly to the course instructor or the administration office to inquire if late enrollment is possible. They can provide specific information on whether you can join the course late and what steps you need to follow.

2. **Check for an Online Option**: Some courses offer online resources, recorded lectures, or flexible deadlines that can accommodate late enrollments.

3. **Understand the Requirements**: If you are allowed to enroll late, make sure you understand any missed material and deadlines. Ask for guidance on how to catch up.

4. **Assess Your Schedule**: Ensure that you have the time and resources to catch up on any missed content or ass

# Applying Prompt Template for FAQ 

In [17]:
prompt_template = """
You are a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT does not contain answer, output NONE.

QUESTION : {question}
CONTEXT : 
{context}
"""

In [18]:
context = ""

for doc in result:
    context = context + f"section : {doc['section']}\nquestion : {doc['question']}\nanswer : {doc['text']}\n\n"

In [19]:
print(context)

section : General course-related questions
question : Course - Can I still join the course after the start date?
answer : Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section : General course-related questions
question : Course - Can I follow the course after it finishes?
answer : Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section : General course-related questions
question : Course - When will the course start?
answer : The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will 

In [20]:
prompt = prompt_template.format(question=q, context=context).strip()
print(prompt)

You are a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT does not contain answer, output NONE.

QUESTION : the course is already started, can I still enroll?
CONTEXT : 
section : General course-related questions
question : Course - Can I still join the course after the start date?
answer : Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section : General course-related questions
question : Course - Can I follow the course after it finishes?
answer : Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on yo

In [21]:
response = client.chat.completions.create(
    model = "gpt-4o",
    messages = [{"role": "user", "content": prompt}]
)
response.choices[0].message.content

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects, so don't leave everything for the last minute."

----------------

## scripting

In [22]:
from openai import OpenAI

client = OpenAI()

In [23]:
def search(q):
    boost = {'question': 3.0, 'section': 0.5,}

    result = index.search(
        query=q,
        filter_dict = {"course" : "data-engineering-zoomcamp"},
        boost_dict=boost,
        num_results=15
    )
    return result


In [24]:
def built_prompt(query, search_results):
    prompt_template = """
    You are a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT does not know answer, output NONE.
    
    QUESTION : {question}
    CONTEXT : 
    {context}
    """
    context = ""
    for doc in search_results:
        context = context + f"section : {doc['section']}\nquestion : {doc['question']}\nanswer : {doc['text']}\n\n"
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [25]:
def llm(prompt):
    response = client.chat.completions.create(
        model = "gpt-4o",
        messages = [{"role": "user", "content": prompt}] 
    )
    return response.choices[0].message.content

In [26]:
def rag(query):
    search_results = search(query)
    prompt = built_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [27]:
# query = "the course just started. can i still enroll?"
# query = "how can I run kafka?"
query = "what are the prerequisites for data engineering zoomcamp?"

answer = rag(query)
answer

'The prerequisites for the Data Engineering Zoomcamp can be found on GitHub at the following link: [GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites](https://github.com/DataTalksClub/data-engineering-zoomcamp#prerequisites).'

# Elastic Search

In [28]:
from elasticsearch import Elasticsearch

In [29]:
es_client = Elasticsearch("http://localhost:9200")

In [30]:
es_client.info()

ObjectApiResponse({'name': '074adecb8bf5', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'NoYKKt1YTd25SawWcQIZRQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
index_name = "course-questions"
es_client.indices.create(index=index_name, body=index_settings)

In [ ]:
from tqdm.auto import tqdm

In [ ]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

In [45]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    return result_docs

In [51]:
query = "the course just started. can i still enroll?"
# query = "how can I run kafka?"
# query = "what are the prerequisites for data engineering zoomcamp?"

In [52]:
result_docs=elastic_search(query)
result_docs

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at

In [53]:
def rag(query):
    search_results = elastic_search(query)
    prompt = built_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [54]:
answer = rag(query)

In [55]:
print(answer)

Yes, you can still enroll and participate in the course even after it has started. You are eligible to submit the homework, but you should be mindful of the deadlines for the final projects.
